In [1]:
# !pip install gymnasium
# !pip install --upgrade swig
# !pip install --upgrade box2D
# !pip3 install box2d box2d-kengz
# !pip install pygame
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import ppo

In [ ]:
policy = ppo.MLP(8,[20],4)
V = ppo.MLP(8,[32,32],1)
env = gym.make("LunarLander-v2", render_mode=None)

policy_optimizer = torch.optim.Adam(policy.parameters(), lr=1e-4)
V_optimizer = torch.optim.Adam(V.parameters(), lr=1e-5)

output_all = []

n_epochs = 100
for i in range(n_epochs):
    output = reward, n_clip, dS, S, advm, advsd, vq = ppo.epoch_train(
        env, policy, V, policy_optimizer, V_optimizer, n_samples = int(1e5), inner_batch_size=int(1e3))
    output_all.append(output)
    
    print("[%0.0f] reward: %0.2f  clip (pct): %0.1f delta S (bps): %0.1f  S: %0.1f advantage mean (bps): %0.1f advantage sd: %0.2f  V quality (pct): %0.3f" % ((i,) + output))

/Users/hollymandel/Documents/llm_curriculum/6-rl/ppo.py:105: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  V_all = V(torch.Tensor(obs_all))[:,0]


[0] reward: -1.93  clip (pct): 0.0 delta S (bps): 1.5  S: 344.3 advantage mean (bps): -46205.7 advantage sd: 18.81  V quality (pct): -0.026
[1] reward: -1.93  clip (pct): 0.1 delta S (bps): 3.3  S: 343.4 advantage mean (bps): -45969.6 advantage sd: 18.79  V quality (pct): -0.026
[2] reward: -1.79  clip (pct): 0.1 delta S (bps): 3.6  S: 342.5 advantage mean (bps): -42280.7 advantage sd: 18.33  V quality (pct): -0.025
[3] reward: -1.65  clip (pct): 0.4 delta S (bps): 5.1  S: 341.0 advantage mean (bps): -39449.4 advantage sd: 18.12  V quality (pct): -0.024
[4] reward: -1.51  clip (pct): 0.5 delta S (bps): 6.7  S: 338.7 advantage mean (bps): -35564.5 advantage sd: 17.58  V quality (pct): -0.025
[5] reward: -1.51  clip (pct): 1.4 delta S (bps): 9.8  S: 334.8 advantage mean (bps): -36126.3 advantage sd: 17.21  V quality (pct): -0.028
[6] reward: -1.43  clip (pct): 1.6 delta S (bps): 11.6  S: 330.0 advantage mean (bps): -34511.7 advantage sd: 16.57  V quality (pct): -0.031
[7] reward: -1.44  

In [ ]:
ppo.plot_ppo_output(output_all)

In [ ]:
env = gym.make("LunarLander-v2", render_mode="human")
obs, info = env.reset()

while True:
    logits = policy.forward(torch.Tensor(obs))
    dist = Categorical(logits=logits)
    action = dist.sample().item()

    obs, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        break